# 1. 들어가며

![https://unsplash.com/photos/mpSeLIXMnpc](https://resources-public-prd.modulabs.co.kr/home-section/story-modulabs-articles-section/857b3112-5c3a-44aa-abba-cd2c72e8843d.png)

이번 시간에는 object detection 모델을 통해 주변에 다른 차나 사람이 가까이 있는지 확인한 후 멈출 수 있는 자율주행 시스템을 만들어 보겠습니다. 하지만 자율주행 시스템은 아직 완전하지 않기 때문에, 위험한 상황에서는 운전자가 직접 운전할 수 있도록 하거나 판단이 어려운 상황에서는 멈추도록 설계됩니다. 우리도 같은 구조를 가진 미니 자율주행 보조장치를 만들어 볼 겁니다.

## 실습 목표
---
  1. PyTorch를 사용하여 Object detection 모델을 학습할 수 있습니다.
  2. RetinaNet 모델을 활용한 시스템을 만들 수 있습니다.

## 학습 내용
---
- 자율주행 보조장치
- RetinaNet (PyTorch)
- 데이터 준비부터 모델 학습, 결과 확인까지
- 프로젝트: 자율주행 보조 시스템 만들기

## 준비물
---
이 실습은 `training_and_evaluation.py` 스크립트에 구현된 PyTorch 기반의 RetinaNet 모델을 사용합니다. 시작하기 전에, KITTI 데이터셋이 TFRecord 형태로 `~/work/object_detection/data/tfrecord/` 경로에 준비되어 있어야 합니다.

아직 경로를 생성하지 않았다면 터미널을 열고 프로젝트를 위한 디렉토리를 생성해 주세요.

```
$ mkdir -p ~/work/object_detection/data/checkpoints
$ mkdir -p ~/work/object_detection/data/tfrecord
# TFRecord 파일을 위 경로로 복사해주세요.
```


# 2. 자율주행 보조장치 
## (1) KITTI 데이터셋

이번 시간에 만들어 볼 자율주행 보조장치는 카메라에 사람이 탐지되었을 때, 그리고 차가 가까워져서 탐지된 크기가 일정 크기 이상일 때를 판단해야 합니다.

> **자율주행 보조장치 object detection 요구사항**
> 1. 사람이 카메라에 감지되면 정지
> 2. 차량이 일정 크기 이상으로 감지되면 정지

![https://www.cvlibs.net/datasets/kitti/](https://resources-public-prd.modulabs.co.kr/home-section/story-modulabs-articles-section/9c16be73-b606-4cad-9922-9d688cd2fe76.png)

이번 시간에는 PyTorch 기반의 커스텀 데이터셋 클래스를 통해 KITTI TFRecord 데이터셋을 사용해보겠습니다. KITTI 데이터셋은 자율주행을 위한 데이터셋으로 2D object detection 뿐만 아니라 깊이까지 포함한 3D object detection 라벨 등을 제공하고 있습니다.

  - [cvlibs에서 제공하는 KITTI 데이터셋](https://www.cvlibs.net/datasets/kitti/)

`training_and_evaluation.py` 스크립트에서는 `tfrecord-torch` 라이브러리를 사용하여 TFRecord 파일을 파싱하고, PyTorch의 `IterableDataset`을 상속받는 `TFRecordKITTIDataset` 클래스를 통해 데이터를 로드합니다.

```python
class TFRecordKITTIDataset(IterableDataset):
    def __init__(self, tfrecord_pattern, transform=None):
        super(TFRecordKITTIDataset, self).__init__()
        self.user_transform = transform
        self.file_paths = sorted([p for p in glob.glob(tfrecord_pattern) if not p.endswith('.index')])
        if not self.file_paths:
            raise FileNotFoundError(f"No TFRecord files found for pattern: {tfrecord_pattern}")

        feature_description = {
            'image': "byte", 'image/file_name': "byte",
            'objects/bbox': "float", 'objects/type': "int",
        }
        
        self.datasets = [
            TFRecordDataset(path, index_path=None, description=feature_description, transform=self._parse_record)
            for path in self.file_paths
        ]
        # ...

    def _parse_record(self, features):
        image = Image.open(io.BytesIO(features['image'])).convert('RGB')
        # ...
        bboxes = torch.tensor(features['objects/bbox'], dtype=torch.float32).view(-1, 4)
        bboxes = bboxes[:, [1, 0, 3, 2]] # y_min, x_min, y_max, x_max -> x_min, y_min, x_max, y_max
        labels = torch.tensor(features['objects/type'], dtype=torch.int64)
        # ...
        sample = {'image': image, 'bbox': bboxes, 'class_id': labels, 'filename': filename}
        
        if self.user_transform:
            sample['image'] = self.user_transform(sample['image'])
            
        return sample
    # ...
```

이 클래스는 지정된 패턴의 TFRecord 파일들을 읽어 이미지와 바운딩 박스, 클래스 라벨을 포함하는 딕셔너리 형태로 반환합니다.

## (2) 데이터 직접 확인하기

데이터셋을 직접 확인해 봅시다. PyTorch의 `DataLoader`를 사용하여 데이터셋을 배치 단위로 불러올 수 있습니다.

```python
# 필요한 클래스와 함수는 training_and_evaluation.py에 있습니다.
# 이 코드를 실행하려면 해당 스크립트의 클래스/함수 정의가 필요합니다.

pil_to_tensor = transforms.Compose([transforms.ToTensor()])
train_dataset = TFRecordKITTIDataset(config.TRAIN_TFRECORD_PATTERN, transform=pil_to_tensor)
train_loader = DataLoader(train_dataset, batch_size=2, collate_fn=collate_fn)

# 데이터 로더에서 한 배치를 가져옵니다.
images, bboxes, class_ids = next(iter(train_loader))

# 첫 번째 이미지와 바운딩 박스를 확인합니다.
img_tensor = images[0]
img_pil = transforms.ToPILImage()(img_tensor)
gt_boxes = bboxes[0]

print("Image shape:", img_tensor.shape)
print("Num boxes:", len(gt_boxes))
```

이미지와 라벨을 얻었으니, 이미지 위에 바운딩 박스(bounding box, bbox)를 그려봅시다. `TFRecordKITTIDataset`은 바운딩 박스를 `[x_min, y_min, x_max, y_max]` 형식의 절대 좌표로 제공합니다.

[Pillow 라이브러리의 ImageDraw 모듈](https://pillow.readthedocs.io/en/stable/reference/ImageDraw.html)을 사용하여 시각화 함수를 작성할 수 있습니다.

<details>
<summary>예시 답안</summary>
<div markdown="1">

```python
from PIL import Image, ImageDraw
import copy

def visualize_bbox(input_image, object_bboxes):
    # input_image는 PIL Image 객체여야 합니다.
    input_image = copy.deepcopy(input_image)
    draw = ImageDraw.Draw(input_image)

    # 바운딩 박스 그리기
    for bbox in object_bboxes:
        # bbox는 [x_min, y_min, x_max, y_max] 형식입니다.
        draw.rectangle(list(bbox), outline=(255,0,0), width=2)

    return input_image

# 첫 번째 샘플 시각화
visualize_bbox(img_pil, gt_boxes)
```

</div>
</details>

# 3. RetinaNet

  - [Focal Loss for Dense Object Detection](https://arxiv.org/abs/1708.02002)

RetinaNet은 **Focal Loss for Dense Object Detection** 논문을 통해 공개된 detection 모델입니다.

1-stage detector 모델인 YOLO와 SSD는 2-stage detector인 Faster-RCNN 등보다 속도는 빠르지만 성능이 낮은 문제를 가지고 있었습니다. 이를 해결하기 위해서 RetinaNet에서는 **focal loss**와 **FPN(Feature Pyramid Network)** 를 적용한 네트워크를 사용합니다.

## Focal Loss
---
>물체를 배경보다 더 잘 학습하자 == 물체인 경우 Loss를 작게 만들자

![https://www.jeremyjordan.me/object-detection-one-stage/](https://resources-public-prd.modulabs.co.kr/home-section/story-modulabs-articles-section/a99458c6-a7b8-4b28-bf82-85343615a79f.png)

Focal loss는 기존의 1-stage detection 모델들(YOLO, SSD)이 물체 전경과 배경을 담고 있는 모든 그리드(grid)에 대해 한 번에 학습됨으로 인해서 생기는 클래스 간의 불균형을 해결하고자 도입되었습니다. 여기서 그리드(grid)와 픽셀(pixel)이 혼란스러울 수 있겠는데, 위 그림 왼쪽 7x7 feature level에서는 한 픽셀이고, 오른쪽의 image level(자동차 사진)에서 보이는 그리드는 각 픽셀의 receptive field입니다.

그림에서 보이는 것처럼 우리가 사용하는 이미지는 물체보다는 많은 배경을 학습하게 됩니다. 논문에서는 이를 해결하기 위해서 Loss를 개선하여 정확도를 높였습니다.

![https://arxiv.org/abs/1708.02002](https://resources-public-prd.modulabs.co.kr/home-section/story-modulabs-articles-section/35768b9e-c523-474f-9aaa-e0edb8798019.png)

Focal loss는 우리가 많이 사용해왔던 교차 엔트로피를 기반으로 만들어졌습니다. 위 그림을 보면 Focal loss는 그저 교차 엔트로피 CE($p_t$)의 앞단에 간단히 ($1−p_t)^γ$라는 modulating factor를 붙여주었습니다.

교차 엔트로피의 개형을 보면 ground truth class에 대한 확률이 높으면 잘 분류된 것으로 판단되므로 손실이 줄어드는 것을 볼 수 있습니다. 하지만 확률이 1에 매우 가깝지 않은 이상 상당히 큰 손실로 이어지는데요.

이 상황은 물체 검출 모델을 학습시키는 과정에서 문제가 될 수 있습니다. 대부분의 이미지에서는 물체보다 배경이 많습니다. 따라서 이미지는 극단적으로 배경의 class가 많은 class imbalanced data라고 할 수 있습니다. 이렇게 너무 많은 배경 class에 압도되지 않도록 modulating factor로 손실을 조절해줍니다. $γ$를 0으로 설정하면 modulating factor ($1−p_t)^γ$가 1이 되어 일반적인 교차 엔트로피가 되고 $γ$가 커질수록 modulating이 강하게 적용되는 것을 확인할 수 있습니다.

## FPN(Feature Pyramid Network)
---
>여러 층의 특성 맵(feature map)을 다 사용해보자

![https://arxiv.org/abs/1612.03144](https://resources-public-prd.modulabs.co.kr/home-section/story-modulabs-articles-section/c9658a6b-dc09-4b78-af8b-3d1cabec6e6b.png)

FPN은 특성을 피라미드처럼 쌓아서 사용하는 방식입니다. CNN 백본 네트워크에서는 다양한 레이어의 결과값을 특성 맵(feature map)으로 사용할 수 있습니다. 이때 컨볼루션 연산은 커널을 통해 일정한 영역을 보고 몇 개의 숫자로 요약해 내기 때문에, 입력 이미지를 기준으로 생각하면 입력 이미지와 먼 모델의 뒷쪽의 특성 맵일수록 하나의 "셀(cell)"이 넓은 이미지 영역의 정보를 담고 있고, 입력 이미지와 가까운 앞쪽 레이어의 특성 맵일수록 좁은 범위의 정보를 담고 있습니다. 이를 **receptive field**라고 합니다. 레이어가 깊어질 수록 pooling을 거쳐 넓은 범위의 정보(receptive field)를 갖게 되는 것입니다.

FPN은 백본의 여러 레이어를 한꺼번에 쓰겠다라는데에 의의가 있습니다. SSD가 각 레이어의 특성 맵에서 다양한 크기에 대한 결과를 얻는 방식을 취했다면 RetinaNet에서는 receptive field가 넓은 뒷쪽의 특성 맵을 upsampling(확대)하여 앞단의 특성 맵과 더해서 사용했습니다. 레이어가 깊어질수록 feature map의 $w, h$방향의 receptive field가 넓어지는 것인데, 넓게 보는 것과 좁게 보는 것을 같이 쓰겠다는 목적인 거죠.

![https://arxiv.org/abs/1708.02002](https://resources-public-prd.modulabs.co.kr/home-section/story-modulabs-articles-section/26edaa1b-4301-49a0-855d-9678bd2615ee.png)

위 그림은 RetinaNet 논문에서 FPN 구조가 어떻게 적용되었는지를 설명하는 그림입니다. RetinaNet에서는 FPN을 통해 $P_3$부터 $P_7$까지의 pyramid level을 생성해 사용합니다. 각 pyramid level은 256개의 채널로 이루어지게 됩니다. 이를 통해 Classification Subnet과 Box Regression Subnet 2개의 Subnet을 구성하게 되는데, Anchor 갯수를 $A$라고 하면 최종적으로 **Classification Subnet**은 $K$개 class에 대해 $KA$개 채널을, **Box Regression Subnet**은 4A개 채널을 사용하게 됩니다.

# 4. 데이터 준비

## 데이터 파이프 라인
---
주어진 KITTI 데이터를 PyTorch 모델 학습에 맞는 형태로 바꾸어 주어야 합니다. `training_and_evaluation.py` 스크립트의 데이터 파이프라인은 다음과 같이 구성됩니다.

1.  **`TFRecordKITTIDataset`**: TFRecord 파일을 읽어 PIL 이미지, 바운딩 박스, 클래스 ID를 반환합니다. 이미지를 Tensor로 변환하는 `transform`을 적용할 수 있습니다.
2.  **`collate_fn`**: `DataLoader`가 배치(batch)를 구성할 때 사용됩니다. 한 배치 내의 이미지들을 가장 큰 이미지 크기에 맞춰 패딩(padding) 처리하여 모든 이미지의 크기를 동일하게 만듭니다.
3.  **`LabelEncoder`**: 배치 단위로 데이터를 받아, 모델이 학습할 수 있는 최종적인 형태(label)로 인코딩합니다. 이 과정에서 Anchor Box 생성, Ground-truth 매칭, 손실 계산을 위한 타겟(target) 생성이 이루어집니다.

## 인코딩
---
One-stage detector인 RetinaNet은 미리 정의된 **Anchor Box**들을 사용합니다. 모델은 이 Anchor Box들을 기준으로 물체의 위치를 예측합니다.

`training_and_evaluation.py`에서는 `AnchorBox` 클래스가 다양한 크기와 비율의 Anchor Box를 생성하고, `LabelEncoder` 클래스가 이 Anchor Box와 실제 정답(Ground-truth) 바운딩 박스를 비교하여 모델이 학습할 타겟을 만듭니다.

`LabelEncoder`의 핵심 로직은 다음과 같습니다.
1.  `_match_anchor_boxes`: 각 Anchor Box와 가장 IoU(Intersection over Union)가 높은 Ground-truth 박스를 찾습니다.
    - IoU가 0.5 이상이면 해당 Anchor Box는 물체가 있는 포지티브(positive) 샘플로 간주됩니다.
    - IoU가 0.4 미만이면 배경(background)인 네거티브(negative) 샘플로 간주됩니다.
    - 0.4와 0.5 사이는 무시(ignore)됩니다.
2.  `_compute_box_target`: 포지티브 샘플로 판정된 Anchor Box에 대해, Ground-truth 박스와의 차이(offset)를 계산합니다. 이 차이가 모델이 예측해야 할 바운딩 박스 회귀(box regression)의 정답(target)이 됩니다.
3.  `encode_batch`: 배치 전체에 대해 위 과정을 수행하여 최종적인 classification/box 타겟 텐서를 생성합니다.

```python
class LabelEncoder:
    def __init__(self):
        self._anchor_box = AnchorBox()
        self._box_variance = torch.tensor(config.BOX_VARIANCE, dtype=torch.float32)

    def _match_anchor_boxes(self, anchor_boxes, gt_boxes, match_iou=0.5, ignore_iou=0.4):
        # ... IoU 계산 및 매칭 ...
        return matched_gt_idx, positive_mask.float(), ignore_mask.float()

    def _compute_box_target(self, anchor_boxes, matched_gt_boxes):
        # ... 박스 회귀 타겟 계산 ...
        return box_target

    def encode_batch(self, batch_images, gt_boxes, cls_ids):
        # ... 배치 단위 인코딩 ...
        return normalized_images, (cls_targets, box_targets)
```

>이 과정에서 `variance`가 등장하는데 관례적으로 Anchor Box를 사용할 때 등장합니다.  
`config.BOX_VARIANCE`는 `[0.1, 0.1, 0.2, 0.2]`로 설정되어 있으며, 이는 각각 x, y, w, h 오프셋의 스케일을 조절하는 역할을 합니다.

이제 데이터를 모델이 학습 가능한 형태로 바꿔 줄 수 있게 되었으니 모델을 만들러 가봅시다.

# 5. 모델 작성

`training_and_evaluation.py` 스크립트는 PyTorch의 `nn.Module`을 사용하여 RetinaNet을 구현합니다. 모델은 크게 Backbone, FPN, Heads 세 부분으로 구성됩니다.

1.  **Backbone (`ResNetBackbone`)**: 사전 학습된 ResNet50을 사용하여 이미지의 특징(feature)을 추출합니다. FPN에서 사용할 여러 레벨의 특성 맵(C3, C4, C5)을 출력합니다.
2.  **FPN (`FeaturePyramid`)**: Backbone에서 나온 특성 맵들을 결합하여 다양한 크기의 객체를 탐지하기 위한 다중 스케일 특성 피라미드(P3-P7)를 구축합니다.
3.  **Heads (`build_head`, `RetinaNet`)**: FPN의 각 레벨 특성 맵에 적용되는 작은 컨볼루션 네트워크입니다. Classification head와 Box regression head가 있으며, 각각 클래스와 바운딩 박스를 예측합니다.

```python
class RetinaNet(nn.Module):
    def __init__(self, num_classes, backbone):
        super(RetinaNet, self).__init__()
        self.fpn = FeaturePyramid(backbone)
        self.num_classes = num_classes
        # Classification Head
        prior_probability = -torch.log(torch.tensor((1 - 0.01) / 0.01))
        self.cls_head = build_head(9 * num_classes, prior_probability)
        # Box Regression Head
        self.box_head = build_head(9 * 4, "zeros")

    def forward(self, image):
        features = self.fpn(image)
        N = image.size(0)
        cls_outputs, box_outputs = [], []
        for feature in features:
            box_outputs.append(self.box_head(feature).view(N, -1, 4))
            cls_outputs.append(self.cls_head(feature).view(N, -1, self.num_classes))
        return torch.cat(cls_outputs, dim=1), torch.cat(box_outputs, dim=1)
```

이제 모델을 준비했고, Loss에 대한 준비를 해봅시다.

RetinaNet에서는 Classification을 위해 **Focal Loss**를, Box Regression을 위해 **Smooth L1 Loss**를 사용합니다. `training_and_evaluation.py`에서는 이 두 손실 함수를 `RetinaNetClassificationLoss`와 `RetinaNetBoxLoss` 클래스로 각각 구현하고, `RetinaNetLoss` 클래스에서 이 둘을 합쳐 최종 손실을 계산합니다.

![https://arxiv.org/pdf/1708.02002.pdf](https://resources-public-prd.modulabs.co.kr/home-section/story-modulabs-articles-section/1bdc1dfc-253a-4893-a953-e382f0e44a32.png)
[Focal Loss + Smooth L1 Loss]  

```python
class RetinaNetLoss(nn.Module):
    def __init__(self):
        super(RetinaNetLoss, self).__init__()
        self._clf_loss = RetinaNetClassificationLoss(config.CLASSIFICATION_LOSS_ALPHA, config.CLASSIFICATION_LOSS_GAMMA)
        self._box_loss = RetinaNetBoxLoss(config.BOX_LOSS_DELTA)
        # ...

    def forward(self, y_pred, y_true):
        cls_preds, box_preds = y_pred
        cls_targets, box_targets = y_true
        
        # ... 마스크 및 정규화 ...
        
        # Classification Loss 계산
        clf_loss = self._clf_loss(one_hot_labels, cls_preds.float())
        # ...
        
        # Box Regression Loss 계산
        box_loss = self._box_loss(box_targets.float(), box_preds.float())
        # ...

        return clf_loss + box_loss
```

이제 모든 준비가 끝났습니다. 모델 학습을 할 수 있겠네요!

# 6. 모델 학습

앞에서 정의한 클래스와 함수들을 사용하여 모델을 조립하고 학습시킵니다. `training_and_evaluation.py`의 `run_training()` 함수가 이 과정을 담당합니다.

학습 과정은 다음과 같습니다.
1.  **장치 설정**: GPU가 사용 가능하면 CUDA를, 아니면 CPU를 사용합니다.
2.  **데이터 로딩**: `TFRecordKITTIDataset`과 `DataLoader`를 사용하여 학습 및 검증 데이터로더를 생성합니다.
3.  **모델, 손실, 옵티마이저 초기화**: `RetinaNet` 모델, `RetinaNetLoss` 손실 함수, `SGD` 옵티마이저를 초기화합니다. Learning rate 스케줄러(`LambdaLR`)도 설정합니다.
4.  **체크포인트 로딩**: 이전에 저장된 체크포인트가 있으면 불러와서 학습을 재개합니다.
5.  **학습 루프**: 정해진 epoch 수만큼 다음을 반복합니다.
    - `model.train()` 모드로 설정하고, 학습 데이터로더에서 배치를 가져옵니다.
    - `LabelEncoder`로 데이터를 인코딩합니다.
    - 모델의 예측을 계산하고(`forward` pass), 손실을 계산합니다.
    - `optimizer.zero_grad()`, `loss.backward()`, `optimizer.step()`을 통해 모델의 가중치를 업데이트합니다.
6.  **검증 루프**: 각 epoch이 끝난 후, `model.eval()` 모드로 설정하고 검증 데이터셋에 대한 손실을 계산하여 모델 성능을 평가합니다.
7.  **체크포인트 저장**: 최신 모델과 검증 손실이 가장 낮은 최적 모델을 `.pth` 파일로 저장합니다.

아래는 `run_training()` 함수의 핵심적인 학습 루프를 간소화한 예시 코드입니다.

```python
# ... (모델, 데이터로더, 옵티마이저 등 초기화) ...

for epoch in range(start_epoch, config.NUM_EPOCHS):
    model.train()
    total_loss = 0
    for i, (batch_images, gt_boxes, cls_ids) in enumerate(train_loader):
        # 1. 데이터 인코딩
        encoded_images, encoded_labels = label_encoder.encode_batch(batch_images, gt_boxes, cls_ids)
        encoded_images = encoded_images.to(device)
        cls_targets, box_targets = encoded_labels[0].to(device), encoded_labels[1].to(device)
        
        # 2. Forward pass
        predictions = model(encoded_images)
        loss = torch.mean(loss_fn(predictions, (cls_targets, box_targets)))

        # 3. Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # ... (검증 및 체크포인트 저장) ...
```

`training_and_evaluation.py` 스크립트를 직접 실행하여 모델 학습을 시작할 수 있습니다.
```bash
python training_and_evaluation.py train
```

# 7. 결과 확인하기

학습된 모델을 불러와 추론 결과를 확인해 봅시다. 추론 시에는 모델의 출력(classification/box 예측)을 사람이 이해할 수 있는 최종적인 바운딩 박스 리스트로 변환하는 디코딩(decoding) 과정이 필요합니다. 이 과정은 `DecodePredictions` 클래스가 담당합니다.

`DecodePredictions`의 역할은 다음과 같습니다.
1.  Anchor Box와 모델의 box 예측값을 결합하여 실제 좌표의 바운딩 박스를 계산합니다.
2.  Classification 예측값에 `sigmoid`를 적용하여 각 클래스에 대한 확률을 얻습니다.
3.  `confidence_threshold` (e.g., 0.05)보다 낮은 확률의 박스들을 제거합니다.
4.  남은 박스들에 대해 NMS(Non-Max Suppression)를 적용하여 겹치는 박스들을 제거합니다.
5.  최종적으로 탐지된 객체들의 바운딩 박스, 점수, 클래스를 반환합니다.

추론을 위한 전체 모델은 학습된 `RetinaNet`과 `DecodePredictions`를 합쳐서 구성할 수 있습니다.

```python
# 1. 학습된 모델 가중치 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RetinaNet(config.NUM_CLASSES, get_backbone()).to(device)

# checkpoint_epoch_10.pth 와 같은 학습된 모델 파일 경로를 지정하세요.
checkpoint = torch.load(config.EPOCH_CHECKPOINT_TEMPLATE.format(epoch=10), map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# 2. 추론할 이미지 준비
# 예시: test 데이터셋의 첫 번째 이미지 사용
test_dataset = TFRecordKITTIDataset(config.TEST_TFRECORD_PATTERN, transform=transforms.Compose([transforms.ToTensor()]))
sample = next(iter(test_dataset))
image_tensor = sample['image'].unsqueeze(0).to(device) # 배치 차원 추가
image_pil = transforms.ToPILImage()(sample['image'])

# 3. 추론 및 디코딩
decoder = DecodePredictions()
with torch.no_grad():
    predictions = model(image_tensor)
    boxes, scores, classes = decoder(image_tensor, predictions)

# 4. 결과 시각화
# visualize_detections 함수는 training_and_evaluation.py에 정의되어 있습니다.
visualize_detections(
    image_pil,
    boxes.cpu().numpy(),
    classes.cpu().numpy(),
    scores.cpu().numpy()
)
```
![https://arxiv.org/pdf/1708.02002.pdf](https://resources-public-prd.modulabs.co.kr/home-section/story-modulabs-articles-section/7a39eb35-65c0-4e6d-8213-443d41e185df.png)
[스크립트의 `DecodePredictions`는 논문의 제안(confidence 0.05, NMS 0.5)을 따릅니다.]

이제 모든 것이 준비 되었으니 학습된 결과를 확인합시다!!

# 8. 프로젝트: 자율주행 보조 시스템 만들기

이제 학습된 RetinaNet 모델을 사용하여 간단한 자율주행 보조 시스템을 만들어 봅시다.

## 1️⃣ 자율주행 시스템 만들기
---
위 7번 섹션의 추론 코드를 바탕으로, 아래의 조건을 만족하는 함수를 만들어 주세요.

  - 입력으로 **PIL Image 객체**를 받습니다.
  - 정지조건에 맞는 경우 "Stop" 아닌 경우 "Go"를 반환합니다.
  - 조건은 다음과 같습니다. (KITTI 데이터셋 라벨 기준: 'Pedestrian'은 3, 'Car'는 0, 'Van'은 1, 'Truck'은 2)
    - 'Pedestrian' (라벨 3) 클래스가 한 명 이상 있는 경우
    - 'Car', 'Van', 'Truck' (라벨 0, 1, 2) 클래스의 크기(width 또는 height)가 300px 이상인 경우

<details>
<summary>프로젝트 구현 가이드</summary>
<div markdown="1">

```python
# 먼저, 모델과 디코더를 초기화하고 학습된 가중치를 로드해야 합니다.
# 이 부분은 한 번만 실행하면 됩니다.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inference_model = RetinaNet(config.NUM_CLASSES, get_backbone()).to(device)
checkpoint = torch.load(config.BEST_CHECKPOINT_FILE, map_location=device) # 가장 성능이 좋았던 모델 사용
inference_model.load_state_dict(checkpoint['model_state_dict'])
inference_model.eval()
decoder = DecodePredictions()
to_tensor = transforms.ToTensor()

def autodrive_system(image_pil):
    """
    자율주행 보조 시스템 함수
    :param image_pil: PIL.Image, 입력 이미지
    :return: "Stop" 또는 "Go"
    """
    
    # 1. 이미지를 텐서로 변환하고 배치 차원을 추가합니다.
    image_tensor = to_tensor(image_pil).unsqueeze(0).to(device)
    
    # 2. 모델 추론 및 디코딩
    with torch.no_grad():
        predictions = inference_model(image_tensor)
        boxes, scores, classes = decoder(image_tensor, predictions)

    # CPU로 데이터 이동
    boxes = boxes.cpu().numpy()
    classes = classes.cpu().numpy()

    # 3. 정지 조건 확인
    # 'Pedestrian' (라벨 3) 확인
    if 3 in classes:
        print("사람이 감지되었습니다.")
        return "Stop"

    # 'Car', 'Van', 'Truck' (라벨 0, 1, 2) 크기 확인
    vehicle_labels = [0, 1, 2]
    for box, cls in zip(boxes, classes):
        if cls in vehicle_labels:
            width = box[2] - box[0]
            height = box[3] - box[1]
            if width >= 300 or height >= 300:
                print(f"큰 차량(class: {cls})이 감지되었습니다.")
                return "Stop"

    return "Go"

# 테스트용 이미지로 함수를 호출해 보세요.
# test_image_pil = Image.open("path/to/your/test_image.jpg")
# result = autodrive_system(test_image_pil)
# print(f"시스템 판단: {result}")
```

</div>
</details>

## 2️⃣ 자율주행 시스템 평가하기
---
아래 `test_system()` 를 통해서 위에서 만든 함수를 평가해봅시다. 제공된 테스트 이미지 10장에 대해 Go와 Stop을 맞게 반환하는지 확인하고 100점 만점으로 평가해줍니다.

```python
def test_system():
    # 테스트 이미지들은 data/go_*.png, data/stop_*.png 에 있습니다.
    test_cases = {
        'data/go_1.png': 'Go', 'data/go_2.png': 'Go', 'data/go_3.png': 'Go', 'data/go_4.png': 'Go', 'data/go_5.png': 'Go',
        'data/stop_1.png': 'Stop', 'data/stop_2.png': 'Stop', 'data/stop_3.png': 'Stop', 'data/stop_4.png': 'Stop', 'data/stop_5.png': 'Stop',
    }
    
    score = 0
    for image_path, expected in test_cases.items():
        try:
            image_pil = Image.open(os.path.join(config.ROOT_DIR, image_path))
            result = autodrive_system(image_pil)
            
            print(f"Testing {image_path}: Expected={expected}, Got={result}")
            
            if result == expected:
                score += 10
        except FileNotFoundError:
            print(f"파일을 찾을 수 없습니다: {image_path}")
        except Exception as e:
            print(f"오류 발생: {e}")

    print(f"
최종 점수: {score} / 100")
    return score

# 평가 실행
test_system()
```
